In [ ]:
import os
import numpy as np
import pandas as pd
import json
from copy import deepcopy
import re
import youdao

In [ ]:
input_path = "XY"
sped_l = os.listdir(input_path)
keyword = json.load(open("../3.KeyWord/keyword.json", "r"))

for sped in sped_l:

    youdao.mk_dir("paper_ld/%s/input" % sped)

    paper_l = os.listdir("%s/%s/outputs" % (input_path, sped))
    paper_l = [ele for ele in paper_l if "ipynb" not in ele]

    for paper in paper_l:

        # 添加翻译
        folder = os.path.exists("paper_ldcn/%s/%s" % (sped, paper))

        if not folder:
            
            youdao.trans(paper, "%s/%s" % (input_path, sped), "paper_ldcn/%s" % (sped))

        with open(
                "%s/%s/%s_pubtator.json" %
            (input_path, sped, paper.replace(".json", "")), "r") as f:
            pubtator_d = json.load(f)
            dis_l = pubtator_d["dis"]
            rm_l = [
                "infection", "infections", "infect", "Infection", "Infections",
                "Infect"
            ]
            dis_l = [ele for ele in dis_l if ele not in rm_l]
            sp_l = pubtator_d["spec"]
            f.close()
        keyword["DISEASE"] = np.unique(dis_l)
        keyword["SPECIES"] = np.unique(sp_l)
        with open("%s/%s/outputs/%s" % (input_path, sped, paper), "r") as f:
            txt = json.load(f)
            full = txt['content']
            f.close()

        anot = {
            'type': 'T',
            'name': '',
            'value': '',
            'start': 0,
            'end': 0,
            'attributes': [],
            'id': 1
        }
        anoa = {'annotation': {'T': [''], 'E': [''], 'R': [''], 'A': ['']}}

        for pre_type in keyword.keys():
            anot["name"] = pre_type
            for word in keyword[pre_type]:
                if word in full:
                    anot["value"] = word
                    for m in re.finditer(
                            word.replace(")", " ").replace("(", " "),
                            full.replace("\n", "").replace(")", " ").replace("(", " ")):
                        anot["start"] = m.start()
                        anot["end"] = m.end()
                        anot["id"] = len(anoa["annotation"]["T"])
                        guodu = deepcopy(anot)
                        anoa["annotation"]["T"].append(guodu)
        txt["outputs"] = anoa
        txt["labeled"] = "true"

#         with open("%s/%s/input/%s" % (input_path, sped, paper), "w") as f:
#             b = json.dumps(txt)
#             f.write(b)